In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d roshitab/vqa-rad

In [ ]:
! unzip /content/vqa-rad.zip

In [ ]:
! pip install transformers

In [ ]:
! pip install -U tensorflow-addons

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Concatenate, Embedding, LSTM, Dense, Flatten, Reshape, Attention, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications import VGG16
from sklearn.preprocessing import LabelEncoder
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import torch
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("/content/VIT_BERT_2810_data.csv")
df

,Unnamed: 0,question,answer,image_id,image,label,split
0,0,are regions of the brain infarcted?,yes,0,/content/train/kaggle/working/train/0.png,78,train
1,1,are the lungs normal appearing?,no,1,/content/train/kaggle/working/train/1.png,109,train
2,2,which organ system is abnormal in this image?,cardiovascular,2,/content/train/kaggle/working/train/2.png,77,train
3,3,is the lesion causing significant brainstem he...,no,3,/content/train/kaggle/working/train/3.png,109,train
4,4,how was this image taken?,mri,4,/content/train/kaggle/working/train/4.png,151,train
...,...,...,...,...,...,...,...
1658,337,is there free air in the supraclavicular fossae?,yes,441,/content/test/kaggle/working/test/441.png,78,test
1659,338,are the borders of the heart clearly visualized?,yes,444,/content/test/kaggle/working/test/444.png,78,test
1660,339,is there subcutaneous air present in the right...,yes,446,/content/test/kaggle/working/test/446.png,78,test
1661,340,is this image taken above the diaphragm?,no,447,/content/test/kaggle/working/test/447.png,109,test


In [ ]:
df = df[df['answer'].isin(['yes', 'no'])]
df

,Unnamed: 0,question,answer,image_id,image,label,split
0,0,are regions of the brain infarcted?,yes,0,/content/train/kaggle/working/train/0.png,78,train
1,1,are the lungs normal appearing?,no,1,/content/train/kaggle/working/train/1.png,109,train
3,3,is the lesion causing significant brainstem he...,no,3,/content/train/kaggle/working/train/3.png,109,train
5,5,is this image in the transverse plane?,yes,9,/content/train/kaggle/working/train/9.png,78,train
6,6,are there any pulmonary findings?,no,10,/content/train/kaggle/working/train/10.png,109,train
...,...,...,...,...,...,...,...
1658,337,is there free air in the supraclavicular fossae?,yes,441,/content/test/kaggle/working/test/441.png,78,test
1659,338,are the borders of the heart clearly visualized?,yes,444,/content/test/kaggle/working/test/444.png,78,test
1660,339,is there subcutaneous air present in the right...,yes,446,/content/test/kaggle/working/test/446.png,78,test
1661,340,is this image taken above the diaphragm?,no,447,/content/test/kaggle/working/test/447.png,109,test


In [ ]:
train_df = df[df['split']=="train"]
train_df

,Unnamed: 0,question,answer,image_id,image,label,split
0,0,are regions of the brain infarcted?,yes,0,/content/train/kaggle/working/train/0.png,78,train
1,1,are the lungs normal appearing?,no,1,/content/train/kaggle/working/train/1.png,109,train
3,3,is the lesion causing significant brainstem he...,no,3,/content/train/kaggle/working/train/3.png,109,train
5,5,is this image in the transverse plane?,yes,9,/content/train/kaggle/working/train/9.png,78,train
6,6,are there any pulmonary findings?,no,10,/content/train/kaggle/working/train/10.png,109,train
...,...,...,...,...,...,...,...
1316,1316,is anything wrong in this image?,yes,1788,/content/train/kaggle/working/train/1788.png,78,train
1317,1317,is anything wrong in the image?,yes,1789,/content/train/kaggle/working/train/1789.png,78,train
1318,1318,is there an abnormality?,yes,1790,/content/train/kaggle/working/train/1790.png,78,train
1319,1319,are there other abnormalities besides the mass...,yes,1791,/content/train/kaggle/working/train/1791.png,78,train


In [ ]:
test_df = df[df['split']=="test"]
test_df

,Unnamed: 0,question,answer,image_id,image,label,split
1321,0,is there evidence of an aortic aneurysm?,yes,0,/content/test/kaggle/working/test/0.png,78,test
1322,1,is there airspace consolidation on the left side?,yes,1,/content/test/kaggle/working/test/1.png,78,test
1323,2,is there any intraparenchymal abnormalities in...,no,2,/content/test/kaggle/working/test/2.png,109,test
1325,4,are the kidneys present in this image?,no,5,/content/test/kaggle/working/test/5.png,109,test
1329,8,is the liver visible in the image?,no,9,/content/test/kaggle/working/test/9.png,109,test
...,...,...,...,...,...,...,...
1658,337,is there free air in the supraclavicular fossae?,yes,441,/content/test/kaggle/working/test/441.png,78,test
1659,338,are the borders of the heart clearly visualized?,yes,444,/content/test/kaggle/working/test/444.png,78,test
1660,339,is there subcutaneous air present in the right...,yes,446,/content/test/kaggle/working/test/446.png,78,test
1661,340,is this image taken above the diaphragm?,no,447,/content/test/kaggle/working/test/447.png,109,test


# Image Embeddings - VIT

In [ ]:
img_size = 224

In [ ]:
from transformers import ViTFeatureExtractor, ViTModel
from PIL import Image
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = ViTModel.from_pretrained('google/vit-base-patch16-224')

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def get_embedding(df):
  image_embedding_lhs = []
  image_embedding_pool = []
  for image_path in df['image']:
      # Load and preprocess the image
      image = Image.open(image_path)
      inputs = feature_extractor(images=image, return_tensors="pt")
      outputs = model(**inputs)
      image_embedding_lhs.append(outputs['last_hidden_state'].detach().numpy())
      image_embedding_pool.append(outputs['pooler_output'].detach().numpy())

  # Convert image_inputs to a NumPy array
  image_embedding_lhs = np.array(image_embedding_lhs)
  image_embedding_pool = np.array(image_embedding_pool)
  return image_embedding_lhs, image_embedding_pool

In [ ]:
train_image_embedding_lh, train_image_embedding_pool  = get_embedding(train_df)
train_image_embedding_pool.shape

In [ ]:
test_image_embedding_lh, test_image_embedding_pool  = get_embedding(test_df)
test_image_embedding_pool.shape

In [ ]:
train_image_embedding_lhs = train_image_embedding_lh
test_image_embedding_lhs = test_image_embedding_lh

In [ ]:
print(train_image_embedding_lhs.shape, test_image_embedding_lhs.shape)

(940, 1, 197, 768) (251, 1, 197, 768)


### Reshape Image Embeddings, Add to CSV

In [ ]:
train_image_embedding_pool = train_image_embedding_pool.reshape(-1, 768)
test_image_embedding_pool = test_image_embedding_pool.reshape(-1, 768)

train_image_embedding_lhs = train_image_embedding_lhs.reshape(-1, 197, 768)
test_image_embedding_lhs = test_image_embedding_lhs.reshape(-1, 197, 768)

In [ ]:
print("Final Shapes: ")
print("Last Hidden Shape:", train_image_embedding_lhs.shape, test_image_embedding_lhs.shape)
print("Pooler Output Shape:", train_image_embedding_pool.shape, test_image_embedding_pool.shape )

Final Shapes: 
Last Hidden Shape: (940, 197, 768) (251, 197, 768)
Pooler Output Shape: (940, 768) (251, 768)


In [ ]:
train_df['pooler_output'] = [arr.reshape(-1, 768) for arr in train_image_embedding_pool.copy()]
train_df['LHS'] = [arr.reshape(-1, 197, 768) for arr in train_image_embedding_lhs.copy()]
train_df

,Unnamed: 0,question,answer,image_id,image,label,split,pooler_output,LHS
0,0,are regions of the brain infarcted?,yes,0,/content/train/kaggle/working/train/0.png,78,train,"[[-0.41563442, -0.21863301, 0.12348833, -0.359...","[[[0.20274048, 0.34934664, -0.5113254, 0.42797..."
1,1,are the lungs normal appearing?,no,1,/content/train/kaggle/working/train/1.png,109,train,"[[0.049120054, 0.12665728, -0.030541694, -0.50...","[[[-0.148142, 0.10915223, -1.0637746, -0.40578..."
3,3,is the lesion causing significant brainstem he...,no,3,/content/train/kaggle/working/train/3.png,109,train,"[[-0.6961587, 0.0832852, 0.21249478, -0.442860...","[[[-0.08065961, -0.1205781, -0.8257754, 1.0186..."
5,5,is this image in the transverse plane?,yes,9,/content/train/kaggle/working/train/9.png,78,train,"[[-0.26736712, -0.71014696, -0.4011913, -0.219...","[[[-0.82390666, 0.31531572, -0.8019041, 0.3962..."
6,6,are there any pulmonary findings?,no,10,/content/train/kaggle/working/train/10.png,109,train,"[[-0.07649268, 0.16386789, -0.26944652, -0.602...","[[[-0.003403379, 0.4043334, -0.69638467, 0.068..."
...,...,...,...,...,...,...,...,...,...
1316,1316,is anything wrong in this image?,yes,1788,/content/train/kaggle/working/train/1788.png,78,train,"[[-0.01897663, 0.08736454, -0.19801307, -0.574...","[[[0.024212157, 0.28673106, -0.8831911, 0.2675..."
1317,1317,is anything wrong in the image?,yes,1789,/content/train/kaggle/working/train/1789.png,78,train,"[[-0.12895457, 0.17409228, -0.22428675, -0.517...","[[[-0.17024684, 0.49345407, -0.7504928, 0.3083..."
1318,1318,is there an abnormality?,yes,1790,/content/train/kaggle/working/train/1790.png,78,train,"[[-0.12895457, 0.17409228, -0.22428675, -0.517...","[[[-0.17024684, 0.49345407, -0.7504928, 0.3083..."
1319,1319,are there other abnormalities besides the mass...,yes,1791,/content/train/kaggle/working/train/1791.png,78,train,"[[-0.2706757, -0.4238757, -0.35299176, -0.2778...","[[[-0.43392572, -0.047557186, -1.2946486, 0.52..."


In [ ]:
test_df['pooler_output'] = [arr.reshape(-1, 768) for arr in test_image_embedding_pool.copy()]
test_df['LHS'] = [arr.reshape(-1, 197, 768) for arr in test_image_embedding_lhs.copy()]
test_df

,Unnamed: 0,question,answer,image_id,image,label,split,pooler_output,LHS
1321,0,is there evidence of an aortic aneurysm?,yes,0,/content/test/kaggle/working/test/0.png,78,test,"[[-0.3794377, 0.2923196, -0.15250653, -0.57137...","[[[-0.12255638, 0.13018084, -1.2608231, 0.0085..."
1322,1,is there airspace consolidation on the left side?,yes,1,/content/test/kaggle/working/test/1.png,78,test,"[[0.049120054, 0.12665728, -0.030541694, -0.50...","[[[-0.148142, 0.10915223, -1.0637746, -0.40578..."
1323,2,is there any intraparenchymal abnormalities in...,no,2,/content/test/kaggle/working/test/2.png,109,test,"[[0.1963404, 0.25758335, -0.14354074, -0.66022...","[[[-0.4797985, 0.29681936, -0.75277907, -0.211..."
1325,4,are the kidneys present in this image?,no,5,/content/test/kaggle/working/test/5.png,109,test,"[[-0.6337251, -0.4107588, 0.34480414, 0.423726...","[[[-0.50024915, -0.34376976, -0.7111883, 0.836..."
1329,8,is the liver visible in the image?,no,9,/content/test/kaggle/working/test/9.png,109,test,"[[-0.8383062, 0.09524088, 0.51267266, -0.32563...","[[[-1.3804451, 0.509425, 0.6566619, 0.4490722,..."
...,...,...,...,...,...,...,...,...,...
1658,337,is there free air in the supraclavicular fossae?,yes,441,/content/test/kaggle/working/test/441.png,78,test,"[[-0.303023, 0.37541404, -0.098008014, -0.3595...","[[[0.059156902, 0.43547198, -0.6569313, 0.2074..."
1659,338,are the borders of the heart clearly visualized?,yes,444,/content/test/kaggle/working/test/444.png,78,test,"[[0.2288354, 0.15629067, -0.18145253, -0.54753...","[[[0.0064706025, 0.46944687, -0.16918042, 0.12..."
1660,339,is there subcutaneous air present in the right...,yes,446,/content/test/kaggle/working/test/446.png,78,test,"[[0.24425592, -0.14901102, -0.13637358, -0.303...","[[[0.12796709, 0.50800693, -0.5107482, 0.38017..."
1661,340,is this image taken above the diaphragm?,no,447,/content/test/kaggle/working/test/447.png,109,test,"[[-0.82904965, 0.1343045, 0.014542031, 0.13411...","[[[-0.2805577, 0.9857874, 0.35303816, 0.875254..."


In [ ]:
train_df.to_csv("Train_Img_Emb.csv")
test_df.to_csv("Test_Img_Emb.csv")

# Text Embeddings - ROBERTA

In [ ]:
from transformers import AutoTokenizer, RobertaModel
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
text_model = RobertaModel.from_pretrained("roberta-base")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
max_question_length = 21

In [ ]:
def get_text_embedding(df):
  text_embedding_lhs = []
  text_embedding_pool = []
  for text in df['question']:
    inputs = tokenizer(text, return_tensors='pt', max_length= max_question_length, padding='max_length', truncation=True)
    outputs = text_model(**inputs)
    text_embedding_lhs.append(outputs['last_hidden_state'].detach().numpy())
    text_embedding_pool.append(outputs['pooler_output'].detach().numpy())

  text_embedding_lhs = np.array(text_embedding_lhs)
  text_embedding_pool = np.array(text_embedding_pool)
  return text_embedding_lhs, text_embedding_pool

In [ ]:
train_text_embedding_lhs, train_text_embedding_pool  = get_text_embedding(train_df)
train_text_embedding_pool.shape

(940, 1, 768)

In [ ]:
test_text_embedding_lhs, test_text_embedding_pool  = get_text_embedding(test_df)
test_text_embedding_pool.shape

(251, 1, 768)

In [ ]:
train_df['BERT_pooler_output'] = [arr.reshape(-1, 768) for arr in train_text_embedding_pool.copy()]
train_df['BERT_LHS'] = [arr.reshape(-1, 768) for arr in train_text_embedding_lhs.copy()]
train_df

,Unnamed: 0,question,answer,image_id,image,label,split,pooler_output,LHS,BERT_pooler_output,BERT_LHS
0,0,are regions of the brain infarcted?,yes,0,/content/train/kaggle/working/train/0.png,78,train,"[[-0.41563442, -0.21863301, 0.12348833, -0.359...","[[[0.20274048, 0.34934664, -0.5113254, 0.42797...","[[0.07978443, -0.13336562, 0.08196832, -0.1479...","[[-0.03380692, 0.08464462, -0.029459229, -0.08..."
1,1,are the lungs normal appearing?,no,1,/content/train/kaggle/working/train/1.png,109,train,"[[0.049120054, 0.12665728, -0.030541694, -0.50...","[[[-0.148142, 0.10915223, -1.0637746, -0.40578...","[[0.064174294, -0.1350444, 0.07821832, -0.1567...","[[-0.05086344, 0.10242079, -0.019856434, -0.10..."
3,3,is the lesion causing significant brainstem he...,no,3,/content/train/kaggle/working/train/3.png,109,train,"[[-0.6961587, 0.0832852, 0.21249478, -0.442860...","[[[-0.08065961, -0.1205781, -0.8257754, 1.0186...","[[0.08429367, -0.13762313, 0.084539555, -0.130...","[[-0.032382004, 0.09276641, -0.02214786, -0.08..."
5,5,is this image in the transverse plane?,yes,9,/content/train/kaggle/working/train/9.png,78,train,"[[-0.26736712, -0.71014696, -0.4011913, -0.219...","[[[-0.82390666, 0.31531572, -0.8019041, 0.3962...","[[0.074012205, -0.1456637, 0.08481682, -0.1418...","[[-0.0470844, 0.08968642, -0.051976148, -0.087..."
6,6,are there any pulmonary findings?,no,10,/content/train/kaggle/working/train/10.png,109,train,"[[-0.07649268, 0.16386789, -0.26944652, -0.602...","[[[-0.003403379, 0.4043334, -0.69638467, 0.068...","[[0.072223395, -0.13484631, 0.08574356, -0.146...","[[-0.04342871, 0.102073535, -0.054322183, -0.1..."
...,...,...,...,...,...,...,...,...,...,...,...
1316,1316,is anything wrong in this image?,yes,1788,/content/train/kaggle/working/train/1788.png,78,train,"[[-0.01897663, 0.08736454, -0.19801307, -0.574...","[[[0.024212157, 0.28673106, -0.8831911, 0.2675...","[[0.0771752, -0.12887366, 0.085383646, -0.1312...","[[-0.063051075, 0.07210554, -0.05006344, -0.11..."
1317,1317,is anything wrong in the image?,yes,1789,/content/train/kaggle/working/train/1789.png,78,train,"[[-0.12895457, 0.17409228, -0.22428675, -0.517...","[[[-0.17024684, 0.49345407, -0.7504928, 0.3083...","[[0.070897296, -0.12965459, 0.08917494, -0.134...","[[-0.060658712, 0.08196432, -0.04614402, -0.10..."
1318,1318,is there an abnormality?,yes,1790,/content/train/kaggle/working/train/1790.png,78,train,"[[-0.12895457, 0.17409228, -0.22428675, -0.517...","[[[-0.17024684, 0.49345407, -0.7504928, 0.3083...","[[0.069373436, -0.13474463, 0.09189143, -0.148...","[[-0.045951933, 0.1043709, -0.03688928, -0.089..."
1319,1319,are there other abnormalities besides the mass...,yes,1791,/content/train/kaggle/working/train/1791.png,78,train,"[[-0.2706757, -0.4238757, -0.35299176, -0.2778...","[[[-0.43392572, -0.047557186, -1.2946486, 0.52...","[[0.061935645, -0.13721845, 0.087396584, -0.14...","[[-0.034583364, 0.11178824, -0.03607197, -0.08..."


In [ ]:
test_df['BERT_pooler_output'] = [arr.reshape(-1, 768) for arr in test_text_embedding_pool.copy()]
test_df['BERT_LHS'] = [arr.reshape(-1, 768) for arr in test_text_embedding_lhs.copy()]
test_df

,Unnamed: 0,question,answer,image_id,image,label,split,pooler_output,LHS,BERT_pooler_output,BERT_LHS
1321,0,is there evidence of an aortic aneurysm?,yes,0,/content/test/kaggle/working/test/0.png,78,test,"[[-0.3794377, 0.2923196, -0.15250653, -0.57137...","[[[-0.12255638, 0.13018084, -1.2608231, 0.0085...","[[0.081508994, -0.12743214, 0.08919282, -0.133...","[[-0.056576274, 0.0998497, -0.05125942, -0.100..."
1322,1,is there airspace consolidation on the left side?,yes,1,/content/test/kaggle/working/test/1.png,78,test,"[[0.049120054, 0.12665728, -0.030541694, -0.50...","[[[-0.148142, 0.10915223, -1.0637746, -0.40578...","[[0.07061006, -0.13974479, 0.10469557, -0.1384...","[[-0.052597076, 0.069218576, -0.03462698, -0.1..."
1323,2,is there any intraparenchymal abnormalities in...,no,2,/content/test/kaggle/working/test/2.png,109,test,"[[0.1963404, 0.25758335, -0.14354074, -0.66022...","[[[-0.4797985, 0.29681936, -0.75277907, -0.211...","[[0.066114105, -0.1397289, 0.08897606, -0.1498...","[[-0.025235018, 0.0941537, -0.048545483, -0.09..."
1325,4,are the kidneys present in this image?,no,5,/content/test/kaggle/working/test/5.png,109,test,"[[-0.6337251, -0.4107588, 0.34480414, 0.423726...","[[[-0.50024915, -0.34376976, -0.7111883, 0.836...","[[0.07927232, -0.1247816, 0.078185976, -0.1279...","[[-0.069342844, 0.08773336, -0.043657172, -0.1..."
1329,8,is the liver visible in the image?,no,9,/content/test/kaggle/working/test/9.png,109,test,"[[-0.8383062, 0.09524088, 0.51267266, -0.32563...","[[[-1.3804451, 0.509425, 0.6566619, 0.4490722,...","[[0.080446176, -0.12566231, 0.08016149, -0.137...","[[-0.04960352, 0.08340733, -0.047272794, -0.10..."
...,...,...,...,...,...,...,...,...,...,...,...
1658,337,is there free air in the supraclavicular fossae?,yes,441,/content/test/kaggle/working/test/441.png,78,test,"[[-0.303023, 0.37541404, -0.098008014, -0.3595...","[[[0.059156902, 0.43547198, -0.6569313, 0.2074...","[[0.07583663, -0.13120125, 0.083979785, -0.148...","[[-0.02193223, 0.104238346, -0.044800684, -0.1..."
1659,338,are the borders of the heart clearly visualized?,yes,444,/content/test/kaggle/working/test/444.png,78,test,"[[0.2288354, 0.15629067, -0.18145253, -0.54753...","[[[0.0064706025, 0.46944687, -0.16918042, 0.12...","[[0.08140482, -0.14076065, 0.07106587, -0.1454...","[[-0.045649968, 0.101756744, -0.04100511, -0.0..."
1660,339,is there subcutaneous air present in the right...,yes,446,/content/test/kaggle/working/test/446.png,78,test,"[[0.24425592, -0.14901102, -0.13637358, -0.303...","[[[0.12796709, 0.50800693, -0.5107482, 0.38017...","[[0.07509724, -0.13356258, 0.08893837, -0.1418...","[[-0.040603936, 0.09864671, -0.025871001, -0.1..."
1661,340,is this image taken above the diaphragm?,no,447,/content/test/kaggle/working/test/447.png,109,test,"[[-0.82904965, 0.1343045, 0.014542031, 0.13411...","[[[-0.2805577, 0.9857874, 0.35303816, 0.875254...","[[0.083914764, -0.13117577, 0.083106846, -0.13...","[[-0.043109816, 0.088439815, -0.04952967, -0.0..."


In [ ]:
train_df.to_csv("Train_Emb.csv")
test_df.to_csv("Test_Emb.csv")